In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import json


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import tensorflow.compat.v1.keras.backend as K
# import tensorflow as tf
# tf.compat.v1.disable_eager_execution()

In [ ]:
# num = int(df[df['image_id'] == '1000015157.jpg']['label'])
# num


# df = df.set_index("image_id")
# int(df.loc[["999329392.jpg"]]['label'])

In [ ]:

# # ---- This works!!!! Converts jpg into array easily
# def parse_image(file):
    


#     img = tf.io.read_file(file)
#     img = tf.image.decode_jpeg(img, channels=3)
    
#     img = tf.image.convert_image_dtype(img, tf.float32)
#     img = tf.image.resize(img, size=(600, 800))

#     return img, image_id

# dataset = tf.data.Dataset.list_files("/kaggle/input/cassava-leaf-disease-classification/train_images/*.jpg")

# dataset = dataset.map(parse_image)

# dataset = dataset.shuffle(20)
# dataset = dataset.batch(16)

# dataset

In [ ]:
# ---- Need to add label
def parse_image(dataset_slice):
    file = dataset_slice['image_id']
    label = dataset_slice['label']
    
#     image_id = str(file).split(".jpg")[0][12:]
#     #image_id = str(file).split("', shape")[0][12:]
    
#     #int(df.loc[[image_id]]['label'])
    
    # now slicing tensor as a string isn't working, but using the file directly seems to be fine
    img = tf.io.read_file("/kaggle/input/cassava-leaf-disease-classification/train_images/" + file)#+ image_id + ".jpg")
    img = tf.image.decode_jpeg(img, channels=3)
    
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, size=(300, 400))
    

    return img, label



df = pd.read_csv("../input/cassava-leaf-disease-classification/train.csv")
dataset = tf.data.Dataset.from_tensor_slices(dict(df))

dataset = dataset.map(parse_image)


# Old way that also worked (except it didn't know the shape)

# from PIL import Image
# def parse_image(file):
#     file = bytes.decode(file.numpy())
    
#     image = tf.constant(np.asarray(Image.open(file)))
#     return image

#dataset = dataset.map(lambda x: tf.py_function(parse_image, [x], [tf.uint8]))


dataset = dataset.shuffle(20)
dataset = dataset.batch(20)
# for item in dataset.take(1):
#     print(str(item['image_id']))
dataset

In [ ]:
next(iter(dataset))

In [ ]:
tensor = next(iter(dataset))[0][0]
tensor

## Model Creation and Training

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation=tf.nn.relu),
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    tf.keras.layers.Dropout(.1),
                                    
                                    tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation=tf.nn.relu),
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    tf.keras.layers.Dropout(.1),
                                    
#                                     tf.keras.layers.Conv2D(filters=64, kernel_size=(3,3), activation=tf.nn.relu),
#                                     tf.keras.layers.MaxPool2D(pool_size=(2,2)),
#                                     tf.keras.layers.Dropout(.1),
                                    tf.keras.layers.Flatten(),
                                    
#                                    tf.keras.layers.Dense(8192, activation=tf.nn.relu),
                                    tf.keras.layers.Dropout(.1),
                                    tf.keras.layers.Dense(1024, activation=tf.nn.relu),
                                    tf.keras.layers.Dropout(.1),
                                    tf.keras.layers.Dense(128, activation=tf.nn.relu),
                                    tf.keras.layers.Dropout(.1),
                                    tf.keras.layers.Dense(5, activation=tf.nn.softmax),
                                    
])
model.build((None, 300, 400, 3))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = model.fit(dataset, epochs=10, verbose=1)

In [ ]:
!pip install -q pyyaml h5py

In [ ]:
model.save('simple_conv2d_cassava')
new_model = tf.keras.models.load_model('simple_conv2d_cassava')


-- the below was just using cpu, using GPU takes 1/10th the time

### First Run
*ETA 67 min for one epoch*
- Batch-size = 16
- image size was (600, 800)
- Simple model, one conv2d, pool, flatten, and 2 dense layers

Max accuracy after 2nd-3rd epoch: ~60%, loss @ 1.222 (see downloaded ss)

### Second Run
Batch-size = 32 --> OOM error

### Third Run
*ETA 47 min for one epoch*
- Batch-size = 20
- image size = (300, 400)
- more complex model, 2 conv2D and pool, flatten and 3 dense layers (1024, 128, 5)
almost done with 5th epoch: accuracy - 94.5%, loss 0.1679


In [ ]:
# old attempts to use dataset


# df = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")
# df['image_id']

# dataset = tf.data.Dataset.from_tensor_slices(dict(df))

# test = dataset.map(parse_image)

#training_generator = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = 0.2, preprocessing_function=parse_image(dataset.take(1)))

# print(dataset.take(1))

# model.fit_generator(generator=training_generator, epochs=5, verbose =1)


# fits the model on batches with real-time data augmentation:
# model.fit(datagen.flow(x_train, y_train, batch_size=32),
#           steps_per_epoch=len(x_train) / 32, epochs=epochs)

# tf.keras.preprocessing.image.ImageDataGenerator(
#     featurewise_center=False,
#     samplewise_center=False,
#     featurewise_std_normalization=False,
#     samplewise_std_normalization=False,
#     zca_whitening=False,
#     zca_epsilon=1e-06,
#     rotation_range=0,
#     width_shift_range=0.0,
#     height_shift_range=0.0,
#     brightness_range=None,
#     shear_range=0.0,
#     zoom_range=0.0,
#     channel_shift_range=0.0,
#     fill_mode="nearest",
#     cval=0.0,
#     horizontal_flip=False,
#     vertical_flip=False,
#     rescale=None,
#     preprocessing_function=None,
#     data_format=None,
#     validation_split=0.0,
#     dtype=None,
# )



#dataset_pixels = dataset.map(parse_image)

# for image in dataset_pixels.take(2):
#     print(image.numpy().decode('utf-8'))



# ------------------------------------------------------------- #

# Way to decode tensor
# for feature_batch in dataset.take(1):
#     print(feature_batch['image_id'].numpy())
#     var = feature_batch['image_id'].numpy().decode('utf-8')
#     print(var)
    
    
# ------------------------------------------------------------- #

    
    
    
    
    
    
    
    
    
#     for key, value in feature_batch.items():

#         print("  {!r:20s}: {}".format(key, value))
        
        
# dataset_process = dataset.map(parse_image)

    
# for pixels in dataset_process.take(2):
#     print(pixels)
        


#df['image_id'] = df['image_id'].str.replace(r'.jpg', '')
#print(type(df['label'][0]))
#dataset = tf.convert_to_tensor(df, dtype=tf.int64)
#tf.convert_to_tensor(my_np_array, dtype=tf.float32)

#type(dataset)

In [ ]:
# from PIL import Image

# # converts to pixeks
# img = np.asarray(Image.open("/kaggle/input/cassava-leaf-disease-classification/train_images/1578977008.jpg"))
# img.shape

# #image_pixels = pd.Series()



# def parse_image(file_name):
#     print(file_name.numpy())
#     print(tf.strings.as_string(file_name['image_id']))
# #    print(file_name['image_id'].as_string())
# #     #print(tf.io.decode_raw(file_name['image_id'], tf.uint8))
    
# # #     unicode_char_bytes = tf.strings.unicode_split(file_name['image_id'], "UTF-8")
# # #     print(unicode_char_bytes)
# #     unicode_values = tf.strings.unicode_decode(file_name['image_id'], "UTF-8")
# #     print(unicode_values)

    
# #     #print(file_name['image_id'].decode('utf-8'))
# #     print(bytes.decode(file_name['image_id']))
#     # get string of filename
#     file = file_name['image_id'].numpy().decode('utf-8')
#     return np.asarray(Image.open("/kaggle/input/cassava-leaf-disease-classification/train_images/" + str(file)))
    

# #image_data_list = parse_image(df['image_id'][0])

In [ ]:
# plt.figure(figsize=(15,10))

# for i in range(1,10):
#     plt.subplot(3,3,i)
#     plt.tight_layout()

#     plt.imshow(df1.iloc[i-1, 1])
#     plt.title(label_dict[str(df1.iloc[i-1, 0])])

#     #plt.colorbar()
#     plt.grid(False)

# plt.show()

convert np array to tensor: ```tf.convert_to_tensor(my_np_array, dtype=tf.float32)```
